# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `Orders` の中で **最も多く注文を行った顧客の `customer_number`** を返す
  *（Follow up: 複数顧客が同数で最大の場合は全員返す）*
* 入力 DF: `Orders`（列: `order_number`, `customer_number`）
* 出力: 列 `customer_number` のみ（順序は任意、重複なし）

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 集約（`groupby.size`）→ 最大件数で抽出。`sort_values` 不要。

```python
import pandas as pd

def find_customer_with_most_orders(orders: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['customer_number']
    """
    # 列最小化（堅牢性のため NULL は除外）
    base = orders[['customer_number']].dropna(subset=['customer_number'])

    if base.empty:
        # 入力が空 or すべて NULL の場合は仕様列のみの空DFを返す
        return pd.DataFrame(columns=['customer_number'])

    # customer_number ごとに件数を計上（ソートしない）
    cnt = base.groupby('customer_number', as_index=False).size()

    # 最大件数
    max_cnt = cnt['size'].max()

    # 最大件数に一致する顧客のみを抽出（同数最大にも対応）
    out = cnt.loc[cnt['size'].eq(max_cnt), ['customer_number']].reset_index(drop=True)

    return out

Analyze Complexity
Runtime 353 ms
Beats 6.62%
Memory 67.14 MB
Beats 65.39%

```

* 返却列は **`customer_number`** のみ。
* `sort_values` を使わずに最大比較で抽出しているため、順序は任意条件を満たします。

## 3) アルゴリズム説明

* 使用 API

  * `DataFrame.dropna(subset=...)`: 不要な NULL 行の除外
  * `DataFrame.groupby(...).size()`: グループ件数の軽量集計
  * ブールインデクシング（`eq`）: 最大件数一致でのフィルタ
  * `reset_index(drop=True)`: 返却整形（列順は `customer_number` のみ）
* **NULL / 重複 / 型**

  * `customer_number` が NULL の行は集計対象外（SQL 仕様を意識した前処理）
  * `groupby.size()` は重複を自然に件数に含める（`order_number` の一意性に依存しない）
  * 返却はユニーク顧客のみで、型は元列に準拠

## 4) 計算量（概算）

* `groupby.size`: **O(N)**（ハッシュ集計想定、グループ数を M とするとメモリは O(M)）
* 最大値比較・フィルタ: **O(M)**

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Orders] --> B[列最小化 customer_number のみ]
  B --> C[groupby.size で件数集計]
  C --> D[最大件数を算出]
  D --> E[最大件数に一致する行を抽出]
  E --> F[出力 customer_number]
```

さらに速くするなら、**`groupby.size()` をやめて “1 列の Series に絞り込み → ソートなし `value_counts` or `numpy.bincount`”** が効きます。特に **`factorize + bincount`** は Pandas 2.2 系でかなり速いです。

以下、**同じシグネチャ**で置き換え候補を 2 つ提示します。

---

## 改訂版A（汎用・簡潔：`value_counts(sort=False)`）

```python
import pandas as pd

def find_customer_with_most_orders(orders: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['customer_number'] のみ。最大同率が複数でも全件返す
    """
    # 1列のSeriesに限定（DataFrameの中間コピーを避ける）
    s = orders['customer_number'].dropna()
    if s.empty:
        return pd.DataFrame(columns=['customer_number'])

    # ソートを完全にスキップ（デフォはソートありで遅い）
    vc = s.value_counts(sort=False)              # index=顧客ID, values=件数（未ソート）
    mx = int(vc.max())
    winners = vc.index[vc.values == mx]

    # 仕様列のみ
    return pd.DataFrame({'customer_number': winners})

Analyze Complexity
Runtime 276 ms
Beats 80.46%
Memory 66.80 MB
Beats 90.95%

```

**ポイント**

* `value_counts(sort=False)` で**内部ソートを回避**（ここが効きます）
* 1 列の Series だけ扱うことで**メモリアロケーション削減**
* タイは `== mx` で一括抽出（`sort_values` 不要）

---

## 改訂版B（最速志向：`factorize + numpy.bincount`）

> 入力の `customer_number` が **任意の型（int/str 混在でも可）**でも動きます。内部で符号化（コード化）してから **`np.bincount`** で超高速カウントします。

```python
import pandas as pd
import numpy as np

def find_customer_with_most_orders(orders: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['customer_number'] のみ。最大同率が複数でも全件返す
    """
    s = orders['customer_number'].dropna()
    if s.empty:
        return pd.DataFrame(columns=['customer_number'])

    # factorize: 値 -> 連番コード（0..K-1）, uniquesは元の値
    codes, uniques = pd.factorize(s, sort=False)   # sort=False で追加ソート回避
    # 連番コードの頻度を一気に数える（最速）
    cnt = np.bincount(codes)                       # shape=(K,)
    mx = int(cnt.max())
    winner_pos = np.flatnonzero(cnt == mx)         # 同率最大のコード位置
    winners = uniques.take(winner_pos)             # 元の顧客番号へ復元

    return pd.DataFrame({'customer_number': winners})

Analyze Complexity
Runtime 280 ms
Beats 76.08%
Memory 66.74 MB
Beats 90.95%

```

**なぜ速い？**

* `factorize` は C 実装のハッシュ化で**ユニーク抽出**が高速
* その後は **連番整数**への `np.bincount` で**純 NumPy の O(N)** カウント
* ソートを一切しない（`idxmax` や `nlargest` 不要）

---

## 追加の実務チューニング

* **dtype を軽量化**：`customer_number` が数値なら `Int32` / `int32` へ（オブジェクト列は遅い）

  ```python
  # 読み込み時 or 前処理時に（例）
  orders['customer_number'] = pd.to_numeric(orders['customer_number'], errors='coerce').astype('Int32')
  ```
* **不要列を渡さない**：上位の呼び出し側で `orders[['customer_number']]` にして渡すと更に僅かに有利
* **欠損が無い前提なら `dropna()` を省略**（分岐コスト削減）

---

## 計算量（改訂版）

* 改訂A：`value_counts(sort=False)` はハッシュ集計で **O(N)**、最大抽出・フィルタは **O(U)**（U=ユニーク顧客数）
* 改訂B：`factorize` **O(N)** → `bincount` **O(N)** → 最大・比較 **O(U)**
  いずれも**ソートなし**なので、元実装より**安定して低レイテンシ**になりやすいです。

---

## まとめ

* 手軽に速く：**改訂A（`value_counts(sort=False)`）**
* とにかく速く：**改訂B（`factorize + np.bincount`）**
  大規模でも伸びがよく、LeetCode などの **Runtime を大幅に縮めやすい**構成です。

